## Identify Points for StreamStats

__Description__: Tool to identify confluence pair points for tributaries of a specific length, add points to the main stem of a stream network at a specific distance interval, and export a shapefile of the points. For additional details, see the [StreamStats Automation Wiki](https://github.com/Dewberry/usgs-tools/wiki/StreamStats-Automation).

__Input__: Stream grid from the [SteamStats Repository](https://streamstatsags.cr.usgs.gov/StreamGrids/directoryBrowsing.asp), masked using `ClipRaster_withMask.ipynb` and the latitude and longitude of the catchment outlet.

__Output__: A shapefile containing the latitude and longitude of points on the stream grid (confluence and main stem locations).


*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [1]:
import os
import sys
sys.path.append('scripts')
from StreamStats_Points import*
import ID_Confluences
import Calc_Distance

SyntaxError: invalid syntax (StreamStats_Points.py, line 93)

### Load the clipped stream grid, specify the catchment outlet, and the tributary exclusion distance:

##### Specify:

In [2]:
path=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results' #Load the stream grid raster which was masked by the catchment polygon
name='Rock_Creek.tif'

lat=1925315.186 #[m]; latitude of the catchment outlet
lon=1616784.964 #[m]; longitude of the catchment outlet 

disexl=(5280/2.0)*(0.3048) #Tributary exclusion distance

##### Load the stream grid:

In [3]:
sg = StreamGrid(os.path.join(path, name)) #Open the stream grid raster and create an object

crs=sg.crs_value() #Extract the coordinate reference system value (epsg) for the raster
print("epsg:",crs) 

df = sg.dataframe() #Create a dataframe from the stream grid data
df.replace(255, 0, inplace=True) #Replace 255 with 0, where 255 corresponds to the non-stream cells
df.head(n=2) 

epsg: 5070


,0,1,2,3,4,5,6,7,8,9,...,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Convert the catchment outlet's lat/lon to row/column in the stream grid dataframe and extract the cell size:

In [4]:
pix_x, pix_y =coord2index(sg, lat, lon) #Transform the lat and lon values to the row/column location with the stream grid dataframe
pourpoint=[(pix_x, pix_y)] #Add these values to a list as a touple
print("Pourpoint XY:", pourpoint)

cellsize=sg.cell_size() #Raster cell size in meters
print("The Cell Size:", cellsize)

Pourpoint XY: [(877, 1848)]
The Cell Size: 10.0


### Move up the stream and identify the confluences:

In [5]:
save_confluence, nogo=ID_Confluences.main(pourpoint, df)

print("Number of points identified:", len(save_confluence))

Number of points identified: 614


### Remove superflous confluences:

In [ ]:
true_confluence, false_confluence=Remove_False_Confluence(save_confluence) #A false confluence is where there are not two points with the same confluence number

print("Number of true confluence points:", len(true_confluence))

##### Identify the original superflous confluence location:

In [ ]:
false_points=ID_False_ConfluenceLocs(false_confluence, nogo)

### Calculate the tributary length:

In [ ]:
tributary, mainstem, nogoabs=Calc_Distance.main(df, cellsize, true_confluence, false_points)
print(len(tributary))

In [ ]:
tributary=[] 
mainstem=[]

walk_confluence=true_confluence.copy() #Copy the true_confluence list, since we want to walk upstream of these to calculate the length to the end of the trib or next confluence
nogo=[walk_confluence[0]]
starting_point=walk_confluence[0] #Assign the first confluence point to the starting_point

total_dis=0.0 #The total distance from the confluence point
count=1
repeat=0

false_pointswocnum=remove_cnum(false_points)

In [ ]:
nogoabs=nogo.copy()

while len(walk_confluence)>0:
        next_cell=MoveUpstream(df, starting_point, nogo)
        
        if len(next_cell)==1 or len(next_cell)>1 and count==1:
            step_dis=TrueDistance(starting_point, next_cell[0], cellsize)
            total_dis=step_dis+total_dis
            nogo.append(next_cell[0])
            starting_point = next_cell[0]
            count+=1
            continue
 
        elif len(next_cell)>1 and 1<count<=4:
            next_cellwocnum=remove_cnum(next_cell)
            if any(x in next_cellwocnum for x in false_pointswocnum): #If any of the cells in next_cell are in false_confluence, then find that cell and assign it to next cell
                nogo=nogo+next_cell
                for cell in next_cell:
                    test_cell=MoveUpstream(df, cell, nogo)
                    if len(test_cell)==0:
                        next_cell.remove(cell)
                step_dis=TrueDistance(starting_point, next_cell[0], cellsize)
                total_dis=step_dis+total_dis
                starting_point=next_cell[0]
                count+=1
                continue
            else:
                if repeat==0:
                    total_dis=0.0
                    starting_point = walk_confluence[0]
                    count=1
                    repeat=1
                else:
                    step_dis=TrueDistance(starting_point, next_cell[0], cellsize)
                    total_dis=step_dis+total_dis
                    mainstem.append(walk_confluence[0]+(total_dis,))
                    walk_confluence.remove(walk_confluence[0])
                    if len(walk_confluence)>0:
                        total_dis=0.0
                        nogoabs=nogoabs+nogo
                        starting_point=walk_confluence[0]                        
                        nogo=[starting_point]
                        repeat=0
                        count=1
                    else:
                        walk_confluence=[] 
            
        elif len(next_cell)>1 and count>4:    
            next_cellwocnum=remove_cnum(next_cell)
            if any(x in next_cellwocnum for x in false_pointswocnum): #If any of the cells in next_cell are in false_confluence, then find that cell and assign it to next cell
                nogo=nogo+next_cell
                for cell in next_cell:
                    test_cell=MoveUpstream(df, cell, nogo)
                    if len(test_cell)==0:
                        next_cell.remove(cell)
                step_dis=TrueDistance(starting_point, next_cell[0], cellsize)
                total_dis=step_dis+total_dis
                starting_point=next_cell[0]
                count+=1
                continue
            else:
                step_dis=TrueDistance(starting_point, next_cell[0], cellsize)
                total_dis=step_dis+total_dis
                mainstem.append(walk_confluence[0]+(total_dis,))
                walk_confluence.remove(walk_confluence[0])
                if len(walk_confluence)>0:
                    total_dis=0.0
                    nogoabs=nogoabs+nogo
                    starting_point=walk_confluence[0]
                    nogo=[starting_point]
                    repeat=0
                    count=1
                else:
                    walk_confluence=[] 
                    
        elif len(next_cell)==0:
            tributary.append(walk_confluence[0]+(total_dis,))
            walk_confluence.remove(walk_confluence[0])
            if len(walk_confluence)>0:
                total_dis=0.0
                nogoabs=nogoabs+nogo
                starting_point=walk_confluence[0]
                nogo=[starting_point]
                repeat=0
                count=1
            else:
                walk_confluence=[]
                
nogoabs=list(set(nogoabs))                

In [ ]:
print((len(mainstem)+len(tributary)), len(mainstem))

##### Remove confluences with tributaries less than specific length:

In [ ]:
incl_tribs=[]

for cell in tributary:
    if cell[3]>=disexl:
        incl_tribs.append(cell) 

print(len(incl_tribs))        

### Save the results:

##### Extract the confluence number and distance:

In [ ]:
cnum_mainstem=[]
cnum_tributary=[]
dis_tribs=[]

for cell in mainstem:
    cnum_mainstem.append(cell[2])

for cell in tributary:
    cnum_tributary.append(cell[2])
    
for cell in tributary:
    dis_tribs.append(cell[3])        

##### Transform and save as a shapefile:

In [ ]:
lists=[save_confluence, true_confluence, mainstem, tributary, false_points, nogoabs, incl_tribs]
names=['save_confluence', 'true_confluence', 'mainstem', 'tributary', 'false_points', 'nogo', 'incl_tribs']
distance=[[],[],[],dis_tribs,[],[],[]]
cnums=[[],[],cnum_mainstem,cnum_tributary,[],[],[]]

for i in range(len(lists)):
    longitude, latitude=index2coord(sg, lists[i])  #Transform the row/column value from the stream grid dataframe to latitude/longitude for each confluence
    gdf=geodataframe(longitude, latitude, crs, distance[i],cnums[i]) #Store the longitude/latitude for each confluence in a geodataframe
    gdf.to_file(filename = os.path.join(path,'{}.shp'.format(names[i]))) #Export the geodataframe as a shapefule

# End